In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import csv 
import re
from py2neo import *
import random
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from datetime import datetime

In [2]:
#导入文件
pd.set_option('display.max_rows', None)
code = pd.read_csv('instrument_info.csv', dtype=(object))
news = pd.read_csv('ninfotag_202207281734.csv')
news = news.content                                          #导入新闻
hangye = pd.read_csv('股票行业分类20220728.csv',encoding = 'gbk')         
hangye=hangye.replace('.SZ','',regex=True)
hangye=hangye.replace('.BJ','',regex=True)
hangye=hangye.replace('.SH','',regex=True)
name = code.s_info_name
mv_0704 = pd.read_csv('0704-0710市值.csv',dtype = object)
code.set_index(['s_info_code'])
hangye.set_index(['s_info_code'])
code_merged = pd.merge(code,hangye,on=['s_info_code'])
code_merged = code_merged.drop_duplicates(subset=['s_info_code'])
code_merged=code_merged.reset_index(drop=True) 
code = code_merged.s_info_code
name = code_merged.s_info_name
indus = code_merged.s_indus
code_merged = pd.merge(code_merged,mv_0704,on=['s_info_code'])
code_merged

,s_info_code,s_info_name,s_indus_code,s_indus,s_val_mv
0,000001,平安银行,61330101,银行Ⅲ,28393630.6
1,000002,万科A,61180101,房地产开发Ⅲ,22711016.81
2,000004,ST国华,61300202,IT服务,144591.4883
3,000005,ST星源,61160201,环保工程及服务Ⅲ,177380.5308
4,000006,深振业A,61180101,房地产开发Ⅲ,600554.939
5,000007,全新好,61200501,一般物业经营,265824.6349
6,000008,神州高铁,61270501,铁路设备,681294.8598
7,000009,中国宝安,61230101,综合Ⅲ,3459462.845
8,000010,美丽生态,61250301,园林工程Ⅲ,399278.2359
9,000011,深物业A,61180101,房地产开发Ⅲ,772388.9032


In [5]:
code_merged.to_csv('C:\\Users\\Administrator\\Desktop\\code_merged.csv')

In [4]:
##连接neo4j数据库，输入地址、用户名、密码
graph = Graph("http://localhost:7474", auth=("neo4j", "123456"))

In [ ]:
node_matcher = NodeMatcher(graph)                                      
relationship_matcher = RelationshipMatcher(graph)
for n in range(len(name)):
    a = node_matcher.match('stock', stock_code = code_merged.s_info_code[n]).first()
    relationship = list(relationship_matcher.match([a], r_type=None))
    time = str(relationship)
    time_str = re.findall(pattern_time, time)
    time_number_sum = 0
    if len(time_str):
        for l in range(len(time_str)):
            time_number = re.sub("[^0-9]","",time_str[l])
            time_number = int(time_number)
            time_number_sum = time_number_sum + time_number
        a.update({'过去一周新闻出现总次数': time_number_sum})
        a.update({'过去一周新闻新闻共现公司总数': len(time_str)})
        graph.push(a)

In [ ]:
node_matcher = NodeMatcher(graph)                                      
relationship_matcher = RelationshipMatcher(graph)
number_news = {}
for n in range(len(name)):
    a = node_matcher.match('stock', stock_code = code_merged.s_info_code[n]).first()
    number_news[n] = a['过去一周新闻出现总次数']

In [ ]:
number_news = list(number_news)
number_news = pd.DataFrame(number_news)
code_merged = pd.concat([code_merged,number_news])

In [ ]:
#通过代码查询结点
node_matcher = NodeMatcher(graph)                                      
relationship_matcher = RelationshipMatcher(graph)                        #建立查询
code = '002074'                                                        #在此输入股票名称  
a = node_matcher.match('stock', stock_code = code).first()              #结点关系类型（任意关系则为None，注意单引号
relationship = list(relationship_matcher.match([a], r_type=None))      #查询结点关系
print(a['stock_name']) 
print(a['stock_code'])
relationship

In [ ]:
#通过名称查询结点
node_matcher = NodeMatcher(graph)                                      
relationship_matcher = RelationshipMatcher(graph)                        #建立查询
name = '天齐锂业'                                                        #在此输入股票名称  
a = node_matcher.match('stock', stock_name = name).first()              #结点关系类型（任意关系则为None，注意单引号
relationship = list(relationship_matcher.match([a], r_type=None))      #查询结点关系
print(a['stock_name']) 
print(a['stock_code'])
print(a['stock_indus_code'])
relationship

In [21]:
node_matcher = NodeMatcher(graph)                                      
relationship_matcher = RelationshipMatcher(graph)
number_news = {}
number_news_code = {}
for n in range(len(name)):
    a = node_matcher.match('stock', stock_code = code_merged.s_info_code[n]).first()
    number_news_code[n] = a['stock_code']
    number_news[n] = a['过去一周新闻出现总次数']

In [22]:
number_news = DataFrame(number_news,index=[0])
number_news_code = DataFrame(number_news_code,index=[0])

In [17]:
number_news

,0,1,2,3,4,5,6,7,8,9,...,4556,4557,4558,4559,4560,4561,4562,4563,4564,4565
0,569,99,None,28,7,4,15,18,52,12,...,None,13,3,2,13,None,None,34,2,138


In [24]:
number_news.to_excel('C:\\Users\\Administrator\\Desktop\\number.xlsx')
number_news_code.to_excel('C:\\Users\\Administrator\\Desktop\\number_code.xlsx')